In [1]:
import pandas as pd
import json

In [2]:
with open('setup.json', 'r') as file:
  ipArg = json.load(file)

In [3]:
excelv2 = pd.read_excel(ipArg['v2filePath'])
print("v2 excel loaded")



v2 excel loaded


In [4]:
excelv3 = pd.read_excel(ipArg['v3filePath'])
print("v3 excel loaded")

v3 excel loaded


In [5]:


limit=ipArg['rowLimit'] # set this as 0, to compare whole excel
uniquefds = ipArg['uniqueFds']
comparisonfds =ipArg['ComparisionFds']

In [6]:
excelv3 = excelv3.rename(columns={
  'Manufacturer Name':'Mapped Manufacturer Name',
  'Manufacturer Part Number':'Mapped Manufacturer PartNumber',
  'Mapped Manufacturer Name':'Manufacturer Name',
  'Mapped Manufacturer Part Number':'Manufacturer Part Number'})


if limit != 0:
  excelv2 = excelv2.head(limit)

limit = len(excelv2) if limit == 0 else limit
print("Row count : ",limit)

Row count :  15035


In [7]:
missingColumnsv2 = [col for col in uniquefds+comparisonfds if col not in excelv2.columns]
missingColumnsv3 = [col for col in uniquefds+comparisonfds if col not in excelv3.columns]
if missingColumnsv2:
  print("V2 Excel Header Missing",missingColumnsv2)
if missingColumnsv3:
  print("V3 Excel Header Missing",missingColumnsv3)

In [8]:
excelv2 = excelv2[uniquefds + comparisonfds]
excelv3 = excelv3[uniquefds + comparisonfds]

In [9]:
excelv2.set_index(uniquefds, inplace=True)
excelv3.set_index(uniquefds, inplace=True)

In [10]:
diff = pd.DataFrame()
missing = pd.DataFrame(columns=uniquefds)


In [11]:
for index, row_df1 in excelv2.iterrows():
  if index in excelv3.index:  # Check if unique combination exists in excelv3
      row_df2 = excelv3.loc[index, comparisonfds]
      #print(row_df1,row_df2)

      # Compare rows, only store differences
      differences = row_df1[comparisonfds].compare(row_df2.squeeze(), keep_equal=True)
      if not differences.empty:
          # print(differences)
          # break
          for uq, uq_value in zip(uniquefds, index if isinstance(index, tuple) else [index]):
            differences[uq] = uq_value
          diff = pd.concat([diff, differences])
  else:
      # Add to missing if unique combination is not found in excelv3
      # = missing.append(pd.Series(index, index=uniquefds), ignore_index=True)
      missing = pd.concat([missing, pd.DataFrame([index], columns=uniquefds)], ignore_index=True)
  

/tmp/ipykernel_140084/574147671.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
  if index in excelv3.index:  # Check if unique combination exists in excelv3
/tmp/ipykernel_140084/574147671.py:3: PerformanceWarning: indexing past lexsort depth may impact performance.
  row_df2 = excelv3.loc[index, comparisonfds]
/tmp/ipykernel_140084/574147671.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
  if index in excelv3.index:  # Check if unique combination exists in excelv3
/tmp/ipykernel_140084/574147671.py:3: PerformanceWarning: indexing past lexsort depth may impact performance.
  row_df2 = excelv3.loc[index, comparisonfds]
/tmp/ipykernel_140084/574147671.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
  if index in excelv3.index:  # Check if unique combination exists in excelv3
/tmp/ipykernel_140084/574147671.py:3: PerformanceWarning: indexing past lexsort depth may impact performance.
  row_df2 = ex

TypeError: can only compare 'Series' (not 'DataFrame') with 'Series'

In [ ]:
diff.to_excel('compared.xlsx')
if not missing.empty:
  missing.to_excel('missing.xlsx')
  print("Missing components saved as missing.xlsx")

Missing components saved as missing.xlsx
